In [2]:
import numpy as np
import tensorflow as tf
import cv2
import keras
from keras import layers
from keras import optimizers 
from keras.layers import Input, Lambda, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, AlphaDropout
from keras.layers import Conv2D, SpatialDropout2D, GlobalAveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.utils import plot_model
import keras.backend as K 
from keras.applications import resnet, resnet_v2, vgg16, vgg19
from keras.datasets import mnist
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
(x_tr, y_tr), (x_test_norm, y_test_norm) = mnist.load_data()
x_tr = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_tr])
x_test_norm = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_test_norm])

In [4]:
x_tr = x_tr.reshape(60000, 56, 56, 1)
x_test_norm = x_test_norm.reshape(10000, 56, 56, 1)
y_tr = to_categorical(y_tr)
y_test_norm = to_categorical(y_test_norm)

In [5]:
from classification_models.keras import Classifiers

# for tensorflow.keras
# from classification_models.tfkeras import Classifiers

ResNet34, preprocess_input = Classifiers.get('resnet18')
base = ResNet34(input_shape=(56, 56, 1), include_top=False, weights=None, pooling='avg')

last = base.layers[-1]

x = GlobalAveragePooling2D()(last.output)
x = Dense(10, activation = 'softmax')(x)
resnet18 = Model(inputs = base.input, outputs = x)

In [16]:
# # base_model101=resnet.ResNet101(input_shape=(56, 56, 1), include_top=False, weights=None, pooling='avg')
# base_model152=resnet.ResNet152(input_shape=(56, 56, 1), include_top=False, weights=None, pooling='avg')

# # last101 = base_model101.layers[-1]

# last152 = base_model152.layers[-1]

# # x101 = Dense(10, activation = 'softmax')(last101.output)
# x152 = Dense(10, activation = 'softmax', kernel_regularizer=keras.regularizers.l2(0.0001))(last152.output)

# # model101 = Model(inputs=base_model101.input, outputs=x101)
# model152 = Model(inputs=base_model152.input, outputs=x152)

In [9]:
resnet18.compile(keras.optimizers.SGD(lr=0.001, momentum = 0.9, nesterov = True), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [11]:
resnet18.fit(x_tr, y_tr, validation_data = (x_test_norm, y_test_norm), batch_size=128, epochs=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 55s 916us/step - loss: 0.0200 - accuracy: 0.9948 - val_loss: 0.0285 - val_accuracy: 0.9904


In [12]:
resnet18.save('models/resnet34_99.h5')

In [5]:
model = load_model('nodropoutresnet1.h5')

In [19]:
model.evaluate(x_test_norm, y_test_norm)

10000/10000 [==============================] - 10s 1ms/step


[0.02154317032408435, 0.9925000071525574]

from IPython.display import Image 

im = cv2.imread('mnist test number.png')

gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im = cv2.resize(gray, (56, 56), interpolation = cv2.INTER_LINEAR).reshape(1,56,56,1)
pred1 = model.predict(im)
print('The number is: ', np.argmax(pred1))